In [57]:
import pandas as pd
import numpy as np
import requests
import urllib.parse
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
# LISTAR CANDIDATOS

#candidatos = pd.read_csv('gobernadores_regionales.csv')
candidatos = pd.read_csv('vicegobernadores_regionales.csv')

dict = {}
for dni_candidato in candidatos['txDocumentoIdentidad']:
    try:
        url = "https://infogob.jne.gob.pe/Politico/ListarPolitico"
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
        body = {"IdDNI": dni_candidato, "token":"cA6%2B6LtErj8=6L"}
        resp = requests.post(url=url, headers=headers, data=body, verify=False)
        data = resp.json()["Data"][0]["TxRutaPolitico"].split("partidario_",1)[1]
        data_encoded = urllib.parse.quote(data.encode('utf8'))
        dict[dni_candidato] = data_encoded
        print(data_encoded)
    except:
        continue
df = pd.DataFrame(dict.items(), columns=["DNI","URL"])
#df.to_csv("dnis_urls_gobernadores_regionales.csv")
df.to_csv("dnis_urls_vicegobernadores_regionales.csv")

In [3]:
# HISTORIAL PARTIDARIO

from requests_html import HTMLSession
session = HTMLSession()

#candidatos = pd.read_csv('dnis_urls_gobernadores_regionales.csv', index_col=0)
candidatos = pd.read_csv('dnis_urls_vicegobernadores_regionales.csv', index_col=0)

# DUMMY
# candidatos = pd.DataFrame({'DNI': [23821919, 22477150,22508609],
#                    'URL': ["Cx8y5zTyGCE%3D8z", "NsJFh4Uc780c6%2B%400ElOxMA%3D%3DJ4", "aT2peYCtZXE%3D2Y"]})

dict = {}
for dni_candidato, url_candidato in candidatos.itertuples(index=False):
    dict[dni_candidato] = {}
    try:
        body = {"token": "KOlSDC8AUxc=lC"}
        r = session.post(f"https://infogob.jne.gob.pe/Politico/_HistorialFichaPolitico?istrParameters={url_candidato}", verify=False, data=body)

        about = r.html.find('.alias')
        for idx, partido in enumerate(about):
            dict[dni_candidato][idx] = partido.text

    except:
        continue

df = pd.DataFrame.from_dict(dict, orient="index")
#df.to_csv("historial_partidario_gobernadores_regionales.csv")
df.to_csv("historial_partidario_vicegobernadores_regionales.csv")

In [48]:
# PROCESOS ELECTORALES

from requests_html import HTMLSession
session = HTMLSession()

#candidatos = pd.read_csv('dnis_urls_gobernadores_regionales.csv', index_col=0)
candidatos = pd.read_csv('dnis_urls_vicegobernadores_regionales.csv', index_col=0)

# DUMMY
candidatos = pd.DataFrame({'DNI': [23821919, 22477150,22508609],
                    'URL': ["Cx8y5zTyGCE%3D8z", "NsJFh4Uc780c6%2B%400ElOxMA%3D%3DJ4", "aT2peYCtZXE%3D2Y"]})

dict = {}
for dni_candidato, url_candidato in candidatos.itertuples(index=False):
    dict[dni_candidato] = {}
    try:
        body = {"token": "KOlSDC8AUxc=lC"}
        r = session.post(f"https://infogob.jne.gob.pe/Politico/_ProcesosFichaPolitico?istrParameters={url_candidato}", verify=False, data=body)

        about = r.html.find('tr')
        for idx, partido in list(enumerate(about))[1:]:
            dict[dni_candidato][idx] = partido.text

    except:
        continue

df = pd.DataFrame.from_dict(dict, orient="index")


#string_dtypes = df.convert_dtypes().select_dtypes("string")
#df[string_dtypes.columns] = string_dtypes.columns.str.split('/partido').str[0]

#df[df.columns] = df[df.columns].astype(str).str.split('/partido').str[0]
#df.to_csv("procesos_electorales_gobernadores_regionales.csv")
df.to_csv("procesos_electorales_vicegobernadores_regionales.csv")
df

,1,2,3
23821919,ELECCIONES REGIONALES Y MUNICIPALES 2018\nGOBE...,ELECCIONES GENERALES 1995\nCONGRESISTA\nFRENTE...,ELECCIONES GENERALES 1990\nSENADOR\nFRENTE POP...
22477150,ELECCIONES REGIONALES Y MUNICIPALES 2018\nCONS...,NaN,NaN
22508609,ELECCIONES REGIONALES Y MUNICIPALES 2018\nALCA...,ELECCIONES REGIONALES Y MUNICIPALES 2014\nALCA...,NaN


In [16]:
df_nativos = pd.read_csv('base_datos_final.csv')

df_nativos['cantidad_personas'] = df_nativos.groupby('idSolicitudLista')['idSolicitudLista'].transform('count')
df_nativos['cantidad_nativos'] = df_nativos.groupby('idSolicitudLista')['fgNativo'].transform('sum')
df_nativos['porcentaje_nativos'] = (df_nativos['cantidad_nativos']/df_nativos['cantidad_personas']*100).round(2)

df_nativos = df_nativos.drop_duplicates('idSolicitudLista')
df_nativos = df_nativos[df_nativos['txTipoEleccion'].isin(['MUNICIPAL PROVINCIAL','REGIONAL'])]
df_nativos = df_nativos[['idSolicitudLista', 'txDocumentoIdentidad', 'txNombreCompleto', 'txCargoEleccion', 'txDescSexo', 'nuEdad', 'fgNativo', 'txOrganizacionPolitica', 'txDescUbigeo', 'idOrganizacionPolitica', 'txDepartamento', 'txProvincia', 'txTipoEleccion', 'cantidad_personas', 'cantidad_nativos', 'porcentaje_nativos']]

df_nativos.to_csv('base_datos_final_con_nativos.csv')

In [84]:
#TEST

from requests_html import HTMLSession
session = HTMLSession()

dni_candidato = "80202186"
url_candidato = "4W4GsqblPdIc6%2B%400ElOxMA%3D%3D4q"

dict = {}
dict[dni_candidato] = {}

body = {"token": "KOlSDC8AUxc=lC"}
r = session.post(f"https://infogob.jne.gob.pe/Politico/_ProcesosFichaPolitico?istrParameters={url_candidato}", verify=False, data=body)

about = r.html.find('tr')
for idx, partido in list(enumerate(about))[1:]:
    dict[dni_candidato][idx] = partido.text.split('\n',5)[:5]

df = pd.DataFrame.from_dict(dict, orient="columns")
#df.to_csv("80202186.csv")
df = df.apply(pd.Series.explode)
df = df.T
#df['DNI'] = df.index
#df = pd.DataFrame(np.reshape(df.values,(2,5)))
#df = pd.DataFrame(df.values.reshape(2, 5), columns=['1','2','3','4','5'])
#df = pd.DataFrame(df.values.reshape(-1, 5))
df

,1,1,1,1,1,2,2,2,2,2
80202186,ELECCIONES REGIONALES Y MUNICIPALES 2018,ALCALDE DISTRITAL,REGION PARA TODOS,PIURA - AYABACA - PACAIPAMPA,SI,ELECCIONES REGIONALES Y MUNICIPALES 2014,ALCALDE DISTRITAL,REGION PARA TODOS,PIURA - AYABACA - PACAIPAMPA,NO
